In [23]:
import numpy as np
import os

# Set random seed for reproducibility (optional)
np.random.seed(42)

In [24]:
# Define function to read input file
def read_input_file(filename):
    with open(filename, "r") as file:
        data = file.read().strip().split()
    mean_interarrival = float(data[0])
    mean_service = float(data[1])
    num_customers = int(data[2])
    return mean_interarrival, mean_service, num_customers

# List of input files to process
input_files = ["io1.txt", "io2.txt", "io3.txt"]

In [25]:
# Define simulation function
def run_simulation(mean_interarrival, mean_service, num_customers, events_filename, results_filename):
    # Initialize simulation variables
    clock = 0.0
    server_busy = False
    num_in_queue = 0
    
    # Event list
    next_arrival_time = np.random.exponential(mean_interarrival)
    next_departure_time = float('inf')
    
    # Queue management
    queue_customer_ids = []
    queue_arrival_times = []
    
    # Counters and accumulators
    num_customers_delayed = 0
    total_delay = 0.0
    area_num_in_queue = 0.0
    area_server_status = 0.0
    
    # Customer tracking
    customer_arrival_counter = 0
    current_customer_in_service = 0
    
    # Event tracking
    event_counter = 0
    
    # Open output file for events
    events_file = open(events_filename, "w")
    
    # Main simulation loop
    while num_customers_delayed < num_customers or server_busy:
        
        # Determine next event
        if next_arrival_time < next_departure_time:
            next_event_time = next_arrival_time
            next_event_type = 'arrival'
        else:
            next_event_time = next_departure_time
            next_event_type = 'departure'
        
        # Update time-average statistics
        time_since_last = next_event_time - clock
        area_num_in_queue += num_in_queue * time_since_last
        area_server_status += (1 if server_busy else 0) * time_since_last
        
        # Advance clock
        clock = next_event_time
        
        # Process the event
        if next_event_type == 'arrival':
            event_counter += 1
            customer_arrival_counter += 1
            
            # Log arrival event
            events_file.write(f"{event_counter}. Next event: Customer {customer_arrival_counter} Arrival\n")
            
            # Schedule next arrival only if we haven't reached the customer limit
            if customer_arrival_counter < num_customers:
                next_arrival_time = clock + np.random.exponential(mean_interarrival)
            else:
                next_arrival_time = float('inf')  # No more arrivals
            
            if not server_busy:
                # Server is idle, customer goes directly to service
                server_busy = True
                current_customer_in_service = customer_arrival_counter
                num_customers_delayed += 1
                
                # No delay for this customer
                # Schedule departure
                service_time = np.random.exponential(mean_service)
                next_departure_time = clock + service_time
            else:
                # Server is busy, customer joins queue
                num_in_queue += 1
                queue_customer_ids.append(customer_arrival_counter)
                queue_arrival_times.append(clock)
        
        else:  # departure event
            event_counter += 1
            
            # Log departure event
            events_file.write(f"{event_counter}. Next event: Customer {current_customer_in_service} Departure\n")
            events_file.write(f"\n---------No. of customers delayed: {num_customers_delayed}--------\n\n")
            
            if num_in_queue == 0:
                # Queue is empty, server becomes idle
                server_busy = False
                next_departure_time = float('inf')
            else:
                # Serve next customer from queue
                num_in_queue -= 1
                current_customer_in_service = queue_customer_ids.pop(0)
                arrival_time = queue_arrival_times.pop(0)
                
                # Calculate delay
                delay = clock - arrival_time
                total_delay += delay
                
                num_customers_delayed += 1
                
                # Schedule departure
                service_time = np.random.exponential(mean_service)
                next_departure_time = clock + service_time
    
    # Close events file
    events_file.close()
    
    # Calculate performance measures
    avg_delay = total_delay / num_customers_delayed if num_customers_delayed > 0 else 0
    avg_num_in_queue = area_num_in_queue / clock if clock > 0 else 0
    server_utilization = area_server_status / clock if clock > 0 else 0
    
    # Write results to file
    with open(results_filename, "w") as f:
        f.write("----Single-Server Queueing System----\n\n")
        f.write(f"Mean inter-arrival time: {mean_interarrival:.6f} minutes\n")
        f.write(f"Mean service time: {mean_service:.6f} minutes\n")
        f.write(f"Number of customers: {num_customers}\n\n")
        f.write(f"Avg delay in queue: {avg_delay:.6f} minutes\n")
        f.write(f"Avg number in queue: {avg_num_in_queue:.6f}\n")
        f.write(f"Server utilization: {server_utilization:.6f}\n")
        f.write(f"Time simulation ended: {clock:.6f} minutes\n")
    
    return avg_delay, avg_num_in_queue, server_utilization, clock

In [26]:
# Run simulation for all input files
for i, input_file in enumerate(input_files, 1):
    print(f"\n{'='*60}")
    print(f"Running simulation for {input_file}")
    print(f"{'='*60}")
    
    # Read input parameters
    mean_interarrival, mean_service, num_customers = read_input_file(input_file)
    
    print(f"Mean inter-arrival time: {mean_interarrival} minutes")
    print(f"Mean service time: {mean_service} minutes")
    print(f"Number of customers: {num_customers}")
    
    # Define output filenames
    events_file = f"event_orders_{i}.txt"
    results_file = f"results_{i}.txt"
    
    # Run simulation
    avg_delay, avg_num_in_queue, server_util, sim_time = run_simulation(
        mean_interarrival, mean_service, num_customers, events_file, results_file
    )
    
    # Display results
    print(f"\nRESULTS:")
    print(f"  Avg delay in queue: {avg_delay:.6f} minutes")
    print(f"  Avg number in queue: {avg_num_in_queue:.6f}")
    print(f"  Server utilization: {server_util:.6f}")
    print(f"  Time simulation ended: {sim_time:.6f} minutes")
    print(f"\nOutput files: {events_file}, {results_file}")

print(f"\n{'='*60}")
print("All simulations completed!")
print(f"{'='*60}")


Running simulation for io1.txt
Mean inter-arrival time: 1.0 minutes
Mean service time: 0.2 minutes
Number of customers: 10

RESULTS:
  Avg delay in queue: 0.000000 minutes
  Avg number in queue: 0.000000
  Server utilization: 0.091477
  Time simulation ended: 12.562038 minutes

Output files: event_orders_1.txt, results_1.txt

Running simulation for io2.txt
Mean inter-arrival time: 1.0 minutes
Mean service time: 0.5 minutes
Number of customers: 1000

RESULTS:
  Avg delay in queue: 0.472808 minutes
  Avg number in queue: 0.483270
  Server utilization: 0.526704
  Time simulation ended: 978.351328 minutes

Output files: event_orders_2.txt, results_2.txt

Running simulation for io3.txt
Mean inter-arrival time: 0.6 minutes
Mean service time: 1.0 minutes
Number of customers: 100

RESULTS:
  Avg delay in queue: 23.914961 minutes
  Avg number in queue: 21.346423
  Server utilization: 0.978731
  Time simulation ended: 112.032640 minutes

Output files: event_orders_3.txt, results_3.txt

All simu